# Yandex.Music

Comparison of Moscow and St. Petersburg is surrounded by myths. For example:
 * Moscow is a metropolis subject to the rigid rhythm of the working week;
 * Petersburg is a cultural capital, with its own tastes.

Using Yandex.Music data, let's compare the behavior of users in the two capitals.

**Purpose of the study** is to test three hypotheses:
1. User activity depends on the day of the week. Moreover, in Moscow and St. Petersburg this manifests itself in different ways.
2. On Monday morning, certain genres dominate in Moscow, while others dominate in St. Petersburg. Similarly, Friday evenings are dominated by different genres, depending on the city.
3. Moscow and St. Petersburg prefer different genres of music. In Moscow, they listen to pop music more often, in St. Petersburg - Russian rap.

**Research progress**

Data on user behavior will be obtained from the `yandex_music_project.csv` file. Nothing is known about the quality of the data. Therefore, before testing hypotheses, a review of the data is required.

We will check the data for errors and evaluate their impact on the study. Then, at the preprocessing stage, we will look for an opportunity to correct the most critical data errors.
 
Thus, the study will take place in three stages:
 1. Data review.
 2. Data preprocessing.
 3. Hypothesis testing.



## Data review

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('yandex_music_project.csv')

Let's display the first ten rows of the table:

In [3]:
display(df.head(10))

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


Get general information about the table:

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


So the table has seven columns. The data type in all columns is `object`.

According to the data documentation:
* `userID` - user ID;
* `Track` - track name;
* `artist` - artist name;
* `genre` - genre name;
* `City` - user's city;
* `time` - start time of listening;
* `Day` - day of the week.

There are three style violations in the column headings:
1. Lowercase letters are combined with uppercase letters.
2. There are gaps.
3. Names of variables consisting of two or more words are not written in "snake" case.



The number of values in the columns varies. This means there are missing values in the data.


**Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Data preprocessing
Let's fix the style in the column headings, eliminate gaps. Then we check the data for duplicates.

### Heading style
Let's display the column names:

In [5]:
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

Let's bring the names in line with good style:
* we will write several-words-titles in the “snake register”,
* all characters will be lowercase,
* eliminate gaps.

To do this, we will rename the columns like this:
* `'  userID'` → `'user_id'`;
* `'Track'` → `'track'`;
* `'  City  '` → `'city'`;
* `'Day'` → `'day'`.

In [6]:
df = df.rename(columns={'  userID': 'user_id', 'Track': 'track', '  City  ': 'city', 'Day': 'day'})

Let's check the result. To do this, once again display the names of the columns:

In [7]:
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Missing values
First, let's calculate how many missing values are in the table.

In [8]:
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Not all missing values affect the study. So in `track` and `artist` the gaps are not important for our work. It suffices to replace them with explicit notation.

But omissions in `genre` can interfere with the comparison of musical tastes in Moscow and St. Petersburg. So we will have to:
* fill in these gaps with explicit notation,
* estimate how much they will damage the calculations. 

Let's replace the missing values in the `track`, `artist` and `genre` columns with the string `'unknown'`. To do this, create a `columns_to_replace` list, iterate through its elements with a `for` loop, and for each column, replace the missing values:

In [9]:
columns_to_replace = ['track', 'artist', 'genre']
for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')

Make sure there are no gaps in the table. To do this, let's count the missing values again.

In [10]:
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Duplicates
Let's count explicit duplicates in the table:

In [11]:
df.duplicated().sum()

3826

Remove explicit duplicates:

In [12]:
df = df.drop_duplicates().reset_index(drop=True)

In [13]:
df.duplicated().sum()

0

Now let's get rid of the implicit duplicates in the `genre` column. For example, the name of the same genre can be spelled slightly differently. Such errors will also affect the result of the study.

Let's display a list of unique genre names, sorted alphabetically.

In [14]:
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Let's go through the list and find the implicit duplicates of the name `hiphop`. These may be misspelled titles or alternative titles in the same genre.

We will see the following implicit duplicates:
* *hip*,
* *hop*,
* *hip-hop*.

To clear the table of them, write the `replace_wrong_genres()` function with two parameters: 
* `wrong_genres` — list of duplicates,
* `correct_genre` — string with correct value.

The function will correct the `genre` column in the `df` table: it will replace each value from the `wrong_genres` list with a value from `correct_genre`.

In [15]:
def replace_wrong_genres(wrong_genres, correct_genre):
    for wrong_genre in wrong_genres:
        df['genre'] = df['genre'].replace(wrong_genre, correct_genre)

In [16]:
wrong_genres = ['hip', 'hop', 'hip-hop']
correct_genre = 'hiphop'

replace_wrong_genres(wrong_genres, correct_genre)

Let's check that we have replaced the wrong names:

*   hip
*   hop
*   hip-hop

Let's output a sorted list of unique values of the `genre` column:

In [17]:
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

**Conclusions**

Preprocessing showed three problems in the data:

- header style violations,
- missing values,
- duplicates - explicit and implicit.

We've fixed the headers to make the table easier to work with. Without duplicates, the study will become more accurate.

We have replaced missing values with `'unknown'`. It remains to be seen whether the gaps in the `genre` column will harm the study.

Now we can move on to hypothesis testing.

## Hypothesis testing

### Comparison of user behavior of two capitals

The first hypothesis states that users listen to music differently in Moscow and St. Petersburg. Let's check this assumption on the data on three days of the week - Monday, Wednesday and Friday. For this:

* Separate users of Moscow and St. Petersburg
* Let's compare how many tracks each group of users listened to on Monday, Wednesday and Friday.

For training, we first perform each of the calculations separately.

Let's evaluate the activity of users in each city. Let's group the data by city and count the plays in each group.

In [18]:
df.groupby('city')['user_id'].count().to_frame().reset_index()

,city,user_id
0,Moscow,42741
1,Saint-Petersburg,18512


There are more auditions in Moscow than in St. Petersburg. It does not follow from this that Moscow users listen to music more often. There are simply more users in Moscow.

Now let's group the data by day of the week and count the plays on Monday, Wednesday, and Friday. Let's take into account that the data contains information about listening only for these days.

In [19]:
df.groupby('day')['user_id'].count().to_frame().reset_index()

,day,user_id
0,Friday,21840
1,Monday,21354
2,Wednesday,18059


On average, users from the two cities are less active on Wednesdays. But the picture may change if we consider each city separately.

Now let's write a function that combines these two calculations.

Let's create a `number_tracks()` function that will count the plays for a given day and city. It will need two parameters:
* day of the week,
* city name.

In the function, we save the rows of the source table into a variable, which value:
  * in the `day` column is equal to the `day` parameter,
  * in the `city` column is equal to the `city` parameter.

To do this, apply sequential filtering with logical indexing.

Then we count the values in the `user_id` column of the resulting table. The result will be stored in a new variable. Let's return this variable from the function.

In [20]:
def number_tracks(day, city):
    df_filtered = df[df['day'] == day]
    df_filtered = df_filtered[df_filtered['city'] == city]
    track_list_count = df_filtered['user_id'].count()
    return track_list_count

Let's call `number_tracks()` six times, changing the value of the parameters so that we get data for each city on each of the three days.

In [21]:
# number of auditions in Moscow on Mondays
number_tracks(day='Monday', city='Moscow')

15740

In [22]:
# number of auditions in St. Petersburg on Mondays
number_tracks(day='Monday', city='Saint-Petersburg')

5614

In [23]:
# number of auditions in Moscow on Wednesdays
number_tracks(day='Wednesday', city='Moscow')

11056

In [24]:
# number of auditions in St. Petersburg on Wednesdays
number_tracks(day='Wednesday', city='Saint-Petersburg')

7003

In [25]:
# number of auditions in Moscow on Fridays
number_tracks(day='Friday', city='Moscow')

15945

In [26]:
# number of auditions in St. Petersburg on Fridays
number_tracks(day='Friday', city='Saint-Petersburg')

5895

Let's create a table using the `pd.DataFrame` constructor, where
* column names - `['city', 'monday', 'wednesday', 'friday']`;
* data - the results we got with `number_tracks`.

In [27]:
# Results table
data = [['Moscow', 15740, 11056, 15945], 
        ['Saint-Petersburg', 5614, 7003, 5895]
       ]
columns = ['city', 'monday', 'wednesday', 'friday']

results_table = pd.DataFrame(data=data, columns=columns)
results_table

,city,monday,wednesday,friday
0,Moscow,15740,11056,15945
1,Saint-Petersburg,5614,7003,5895


**Conclusions**

The data shows the difference in user behavior:

- In Moscow, the peak of listening falls on Monday and Friday, and on Wednesday there is a noticeable decline.
- In St. Petersburg, on the contrary, they listen to music more on Wednesdays. Activity on Monday and Friday here is almost equally inferior to Wednesday.

So the data support the first hypothesis.

### Music at the beginning and end of the week

According to the second hypothesis, on Monday morning certain genres predominate in Moscow, while others dominate in St. Petersburg. Similarly, Friday evenings are dominated by different genres, depending on the city.

Let's save tables with data in two variables:
* in Moscow - in `moscow_general`;
* in St. Petersburg - in `spb_general`.

In [28]:
moscow_general = df[df['city'] == 'Moscow']

In [29]:
spb_general = df[df['city'] == 'Saint-Petersburg']

Let's create a `genre_weekday()` function with four parameters:
* table (dataframe) with data,
* day of the week,
* start timestamp in 'hh:mm' format,
* last timestamp in 'hh:mm' format.

The function should return information about the top 10 genres of those tracks that were listened to on the specified day, in the interval between two timestamps.

In [30]:
def genre_weekday(table, day, time1, time2):
    genre_df = table[table['day'] == day]
    genre_df = genre_df[genre_df['time'] > time1]
    genre_df = genre_df[genre_df['time'] < time2]
    
    genre_df_count = genre_df.groupby('genre')['genre'].count()
    genre_df_sorted = genre_df_count.sort_values(ascending=False)
    
    return genre_df_sorted[:10]

Let's compare the results of the `genre_weekday()` function for Moscow and St. Petersburg on Monday morning (from 7:00 to 11:00) and Friday evening (from 17:00 to 23:00):

In [31]:
genre_weekday(moscow_general, 'Monday', '07:00', '11:00')

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: genre, dtype: int64

In [32]:
genre_weekday(spb_general, 'Monday', '07:00', '11:00')

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: genre, dtype: int64

In [33]:
genre_weekday(moscow_general, 'Friday', '17:00', '23:00')

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: genre, dtype: int64

In [34]:
genre_weekday(spb_general, 'Friday', '17:00', '23:00')

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: genre, dtype: int64

**Conclusions**

If we compare the top 10 genres on Monday morning, we can draw the following conclusions:

1. In Moscow and St. Petersburg they listen to similar music. The only difference is that the Moscow rating includes the “world” genre, while the St. Petersburg rating includes jazz and classical.

2. There were so many missing values ​​in Moscow that the value `'unknown'` took tenth place among the most popular genres. This means that missing values ​​occupy a significant share in the data and threaten the reliability of the study.

Friday night does not change this picture. Some genres rise a little higher, others go down, but overall the top 10 stays the same.

Thus, the second hypothesis was only partially confirmed:
* Users listen to similar music at the beginning of the week and at the end.
* The difference between Moscow and St. Petersburg is not very pronounced. In Moscow, they listen to Russian popular music more often, in St. Petersburg - jazz.

However, gaps in the data cast doubt on this result. There are so many of them in Moscow that the top 10 ranking could look different if it were not for the lost genre data.

### Genre preferences in Moscow and St. Petersburg

Hypothesis: St. Petersburg is the capital of rap, the music of this genre is listened to more often than in Moscow. And Moscow is a city of contrasts, which, nevertheless, is dominated by pop music.

Let's group the `moscow_general` table by genre and count the listens of tracks of each genre using the `count()` method. Then we sort the result in descending order and store it in the `moscow_genres` table.

In [35]:
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending=False)

Let's display the first ten lines of `moscow_genres`:

In [36]:
moscow_genres[:10]

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Now let's repeat the same for St. Petersburg.

Let's group the `spb_general` table by genre. Let's count listening to tracks of each genre. Sort the result in descending order and store it in the `spb_genres` table:

In [37]:
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending=False)
spb_genres[:10]

genre
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Conclusions**
The hypothesis was partially confirmed:
* Pop music is the most popular genre in Moscow, as the hypothesis suggested. Moreover, in the top 10 genres there is a close genre - Russian popular music.
* Contrary to expectations, rap is equally popular in Moscow and St. Petersburg.

## Results of the study

We tested three hypotheses and found:

1. The day of the week has a different effect on the activity of users in Moscow and St. Petersburg.

The first hypothesis was fully confirmed.

2. Musical preferences do not change much during the week - be it Moscow or St. Petersburg. Small differences are noticeable at the beginning of the week, on Mondays:
* in Moscow they listen to music of the “world” genre,
* in St. Petersburg - jazz and classical music.

Thus, the second hypothesis was only partly confirmed. This result could have been different were it not for gaps in the data.

3. The tastes of users of Moscow and St. Petersburg have more in common than differences. Contrary to expectations, genre preferences in St. Petersburg resemble those in Moscow.

The third hypothesis was not confirmed.